In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader,random_split
from torch.optim import SGD
import os 
import json


In [3]:
annotation_train_dir = "./ann_subsample/ann_subsample"

In [4]:
annotation_train_dir = "./ann_subsample/ann_subsample/"
files = os.listdir(annotation_train_dir)

poses = {}

for pose in files:
    with open(os.path.join(annotation_train_dir,pose)) as f:
        data = json.load(f)
        poses.update({pose.replace('.json',''): data})

In [5]:
data = {

}

for pose in poses.keys():
    data.update({pose:[]})


for pose in poses.keys():
    for image in poses[pose].keys():
        landmarks = poses[pose][image]["landmarks"]
        if len(landmarks[0]) == 21:
            data[pose].append(torch.tensor(landmarks[0]).squeeze().flatten())



In [6]:
for pose in data.keys():
    data.update({pose:torch.stack(data[pose])})



In [112]:
from typing import Dict

class LandmarkSet(Dataset):
    def __init__(self, data:Dict[str,torch.Tensor]):
        x = torch.tensor([])
        y = torch.tensor([])

        for index,pose in enumerate(data.keys()):
            x = torch.concat((x,data[pose]))
            _y = torch.zeros((18))
            _y[index] = 1
            y = torch.concat((y,_y.repeat((len(data[pose]),1))))
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, index) :
        return self.x[index],self.y[index]
    


In [117]:
landmark_set = LandmarkSet(data)
length = len(landmark_set)
train_data,valid_data,test_data= random_split(landmark_set,[int(x * length) for x in [0.8,0.1,0.1]])# 


In [118]:
train_dataloader = DataLoader(train_data, batch_size=128, shuffle=True)
valid_dataloader = DataLoader(valid_data,batch_size=64,shuffle=True)

In [186]:

GestureModel = nn.Sequential(
    nn.Linear(42,30),
    nn.ReLU(),
    nn.Linear(30,21),
    nn.ReLU(),
    nn.Linear(21,18),
    nn.ReLU()
)

In [190]:
EPOCHS = 100
lr = 1e-1
loss_func = nn.CrossEntropyLoss()
opt = SGD(GestureModel.parameters(),lr=lr)
def fit():
    for epoch in range(EPOCHS):
        epoch_train_loss = 0
        epoch_valid_loss = 0
        for batch in train_dataloader:
            xb,yb = batch
            preds = GestureModel(xb)
            loss = loss_func(preds,yb)
            with torch.no_grad():
                epoch_train_loss += loss
            loss.backward()
            opt.step()
            opt.zero_grad()
            
        print(f"Epoch {epoch} train Loss: {epoch_train_loss/len(train_dataloader)}")

        for batch in valid_dataloader:
            with torch.no_grad():
                xb,yb = batch
                preds = GestureModel(xb)
                loss = loss_func(preds,yb)
                with torch.no_grad():
                    epoch_valid_loss += loss
        
        print(f"Epoch {epoch} valid Loss: {epoch_valid_loss/len(valid_dataloader)}")



fit()




Epoch 0 train Loss: 2.890371561050415
Epoch 0 valid Loss: 2.8903720378875732
Epoch 1 train Loss: 2.890371561050415
Epoch 1 valid Loss: 2.8903720378875732
Epoch 2 train Loss: 2.890371561050415
Epoch 2 valid Loss: 2.8903720378875732
Epoch 3 train Loss: 2.890371561050415
Epoch 3 valid Loss: 2.890371561050415
Epoch 4 train Loss: 2.890371561050415
Epoch 4 valid Loss: 2.8903720378875732
Epoch 5 train Loss: 2.890371561050415
Epoch 5 valid Loss: 2.8903720378875732
Epoch 6 train Loss: 2.890371561050415
Epoch 6 valid Loss: 2.8903720378875732
Epoch 7 train Loss: 2.890371561050415
Epoch 7 valid Loss: 2.8903720378875732
Epoch 8 train Loss: 2.890371561050415
Epoch 8 valid Loss: 2.8903720378875732
Epoch 9 train Loss: 2.890371561050415
Epoch 9 valid Loss: 2.890371561050415
Epoch 10 train Loss: 2.890371561050415
Epoch 10 valid Loss: 2.8903720378875732
Epoch 11 train Loss: 2.890371561050415
Epoch 11 valid Loss: 2.8903720378875732
Epoch 12 train Loss: 2.890371561050415
Epoch 12 valid Loss: 2.890371561050

In [156]:
import torch.nn as nn
 
ce = nn.CrossEntropyLoss()
 
logits = torch.tensor([[-1.90, -0.29, -2.30], [-0.29, -1.90, -2.30]])
target = torch.tensor([[0., 1., 0.], [1., 0., 0.]])
logits.shape
target.shape
# print("Cross entropy: %.3f" % ce(logits, target))

torch.Size([2, 3])